# Project work: A mini segmentation challenge

Imaging for the Life Sciences  
MSLS / CO4: Project work

**Student**: $\Rightarrow$ Mirco Blaser

**University**: $\Rightarrow$ ZHAW

**Semester**: $\Rightarrow$ 4th Semester

**Date**: $\Rightarrow$ June 3rd 2024


<br><br><br>
## Table of contents
<!-- Unfortunately, the following does not always work correctly -->
* [1. Dataset](#sec_dataset)  
* [2. Preprocessing](#sec_preprocessing)  
* [3. Manual segmentation](#sec_manual_segmentation)  
* [4. Automated segmentation](#sec_automated_segmentation)  
* [5. Evaluation](#sec_evaluation)  
* [6. Discussion](#sec_discussion)  
* [*. Hints](#sec_hints)  


---

## Prerequisites / Setup

$\Rightarrow$  Special setup instructions, imports and configurations go here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import nibabel as nib
import pydicom
import PIL
from PIL import Image
import os

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

# Inline backend configuration
%matplotlib inline

# Enable this line if you want to use the interactive widgets
# It requires the ipympl package to be installed.
#%matplotlib widget

import sys
sys.path.insert(0, "../")
import tools

# Number of samples to create for the whole code
num_samples = 10

---

<a id='sec_dataset'></a>

## Dataset

Title: Satellite Images of Water Bodies

Source: [Kaggle](https://www.kaggle.com/datasets/franciscoescobar/satellite-images-of-water-bodies)

Description: A collection of water bodies images captured by the Sentinel-2 Satellite. Each image comes with a black and white mask where white represents water and black represents something else but water. The masks were generated by calculating the NWDI (Normalized Water Difference Index) which is frequently used to detect and measure vegetation in satellite images, but a greater threshold was used to detect water bodies.

- **Images**: These are the raw satellite images.
- **Masks**: These are the binary masks where water bodies are labeled.

Below are examples of the images and their corresponding masks:

In [ ]:
# Paths to the directories containing images and masks
images_path = './data/Images'
masks_path = './data/Masks'

# Function to load images and masks
def load_images_masks(image_dir, mask_dir, num_samples):
    image_files = sorted(os.listdir(image_dir))[:num_samples]
    mask_files = sorted(os.listdir(mask_dir))[:num_samples]
    
    images = [Image.open(os.path.join(image_dir, file)) for file in image_files]
    masks = [Image.open(os.path.join(mask_dir, file)) for file in mask_files]
    
    return images, masks

# Load the first num_samples images and masks
images, masks = load_images_masks(images_path, masks_path, num_samples)

# Display the images and masks
fig, axs = plt.subplots(num_samples, 2, figsize=(10, num_samples*5))

for i in range(num_samples):
    axs[i, 0].imshow(images[i])
    axs[i, 0].set_title(f'Image {i+1}')
    axs[i, 0].axis('off')
    
    axs[i, 1].imshow(masks[i], cmap='gray')
    axs[i, 1].set_title(f'Mask {i+1}')
    axs[i, 1].axis('off')

plt.tight_layout()
plt.show()


---

<a id='sec_preprocessing'></a>

## Preprocessing

1. **Resizing**: Standardized the image sizes to 256x256 pixels.
2. **Normalization**: Adjusted the pixel values to the range [0, 1].
3. **Contrast Enhancement**: Applied histogram equalization to enhance the contrast of the images.

Below are the examples of the preprocessed images and their corresponding masks:

In [ ]:
import PIL
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

def preprocess_image(image, ismask):
    """
    Preprocess the input image: resize while maintaining aspect ratio,
    convert to grayscale if it's not a mask.
    
    Args:
    - image (PIL.Image): The input image to preprocess.
    - ismask (bool): Flag to indicate if the image is a mask.
    
    Returns:
    - numpy.ndarray: The preprocessed image.
    """
    target_width = 256
    
    # Calculate the target height to maintain the original aspect ratio
    width, height = image.size
    target_height = int(height * (target_width / width))
    
    # Resize the image while maintaining aspect ratio
    image_resized = image.resize((target_width, target_height), PIL.Image.LANCZOS)
    
    if not ismask:
        
        # Convert the image to a numpy array and then to grayscale directly
        image_array = np.array(image_resized)
        image_grayscale = cv.cvtColor(image_array, cv.COLOR_BGR2GRAY)
        image_enhanced = image_grayscale
    else:
        image_enhanced = np.array(image_resized.convert('L'))
    
    return image_enhanced

# Ensure num_samples does not exceed the length of images or masks
num_samples = min(len(images), len(masks))

# Preprocess the first num_samples images and masks
preprocessed_images = [preprocess_image(images[i], ismask=False) for i in range(num_samples)]
preprocessed_masks = [preprocess_image(masks[i], ismask=True) for i in range(num_samples)]

# Display the preprocessed images and masks
fig, axs = plt.subplots(num_samples, 2, figsize=(10, num_samples*5))

for i in range(num_samples):
    axs[i, 0].imshow(preprocessed_images[i], cmap='gray')
    axs[i, 0].set_title(f'Preprocessed Image {i+1}')
    axs[i, 0].axis('off')
    
    axs[i, 1].imshow(preprocessed_masks[i], cmap='gray')
    axs[i, 1].set_title(f'Preprocessed Mask {i+1}')
    axs[i, 1].axis('off')

plt.tight_layout()
plt.show()


---
<a id='sec_manual_segmentation'></a>

## Manual segmentation

For the manual segmentation, the Fiji (ImageJ) software was used. Fiji is an open-source image processing package that is widely used in the life sciences for its powerful and user-friendly tools.

### Steps for Manual Segmentation in Fiji:

1. **Open Image**: Load the image into Fiji by going to `File > Open...` and selecting the image file.
2. **Select the Region of Interest**: Use the Polygon selection tool to carefully outline the water body in the image.
3. **Create a Mask**: Once the water body is selected, go to Edit > Selection > Create Mask. This creates a binary mask where the selected region is white, and the rest is black.
5. **Save Mask**: Save the resulting mask by going to `File > Save As > PNG...`.

The following code displays the original images, the original masks, and the manually segmented masks for the selected images (100, 170, and 708).

In [ ]:
# Paths to the directories containing images and masks
images_path = './data/Images/'
masks_path = './data/Masks/'
manual_masks_path = './manual/'

# List of specific image indices to load
image_indices = [100, 170, 708]

# Function to load specific images and masks
def load_specific_images_masks(image_dir, mask_dir, manual_mask_dir, indices):
    images = []
    masks = []
    manual_masks = []
    for index in indices:
        image_file = f'water_body_{index}.jpg'
        mask_file = f'water_body_{index}.jpg'
        manual_mask_file = f'water_body_{index}.png'
        image_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, mask_file)
        manual_mask_path = os.path.join(manual_mask_dir, manual_mask_file)
        images.append(Image.open(image_path))
        masks.append(Image.open(mask_path))
        manual_masks.append(Image.open(manual_mask_path))
    
    return images, masks, manual_masks

# Load the specific images and masks
images, masks, manual_masks = load_specific_images_masks(images_path, masks_path, manual_masks_path, image_indices)

# Display the images and masks
fig, axs = plt.subplots(len(images), 3, figsize=(10, 5 * len(images)))

for i in range(len(images)):
    axs[i, 0].imshow(images[i])
    axs[i, 0].set_title(f'Image {image_indices[i]}')
    axs[i, 0].axis('off')
    
    axs[i, 1].imshow(masks[i], cmap='gray')
    axs[i, 1].set_title(f'Original Mask {image_indices[i]}')
    axs[i, 1].axis('off')
    
    axs[i, 2].imshow(manual_masks[i], cmap='gray')
    axs[i, 2].set_title(f'Manually segmented Mask {image_indices[i]}')
    axs[i, 2].axis('off')

plt.tight_layout()
plt.show()


---

<a id='sec_automated_segmentation'></a>

## Automated segmentation

$\Rightarrow$ Describe how to segment the image in Python


### Goals:
* The segmentation must be performed in Python.
* Using an external library or tool (e.g. OpenCV) is permitted.
* Implement a function `segment(image, ...)` takes an image as input and creates a segmentation mask for the structure of interest.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Define the segmentation function
def segment(image, method='global'):
    """
    Segment the image using the specified method.
    
    Args:
    - image (numpy.ndarray): The input image to segment.
    - method (str): The thresholding method to use ('global', 'adaptive', 'otsu', 'watershed').
    
    Returns:
    - numpy.ndarray: The segmentation mask.
    """
    
    # Check the number of channels in the image
    if len(image.shape) == 3 and image.shape[2] == 3:
        # Convert the image to grayscale if it has 3 channels
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    else:
        # Image is already in grayscale
        gray = image
    
    if method == 'global':
        # Apply global thresholding
        _, mask = cv.threshold(gray, 127, 255, cv.THRESH_BINARY)
    elif method == 'adaptive':
        # Apply adaptive thresholding
        mask = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    elif method == 'otsu':
        # Apply Otsu's thresholding
        _, mask = cv.threshold(gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    elif method == 'watershed':
        # Apply watershed segmentation
        markers, img = segment_watershed(image)
        # Create a binary mask where the segmented regions are white
        mask = np.zeros_like(gray)
        mask[markers > 1] = 255  # We assume the labels greater than 1 correspond to foreground regions
    else:
        raise ValueError(f"Unknown method: {method}")
    
    return mask

def segment_watershed(img):
    img = cv.GaussianBlur(img, (5, 5), 0)
    img_blur = cv.medianBlur(img, 5)
    
    # Convert the image to grayscale
    if len(img_blur.shape) == 3 and img_blur.shape[2] == 3:
        gray = cv.cvtColor(img_blur, cv.COLOR_RGB2GRAY)
    else:
        gray = img_blur
    
    ret, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # Noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=9)

    # Sure background area
    sure_bg = cv.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
    thr = 18
    ret, sure_fg = cv.threshold(dist_transform, thr, 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg, sure_fg)

    # Marker labelling
    ret, markers = cv.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0
    
    # Ensure the image is in color
    if len(img.shape) == 2 or img.shape[2] != 3:
        img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)

    markers = cv.watershed(img, markers)
    img[markers == -1] = [255, 0, 0]

    return markers, img

# Segment the preprocessed images using different methods
segmented_masks_otsu = [segment(image, method='otsu') for image in preprocessed_images]
segmented_masks_adaptive = [segment(image, method='adaptive') for image in preprocessed_images]
segmented_masks_global = [segment(image, method='global') for image in preprocessed_images]
segmented_masks_watershed = [segment(image, method='watershed') for image in preprocessed_images]

# Display the results
fig, axs = plt.subplots(num_samples, 6, figsize=(20, num_samples * 5))

for i in range(num_samples):
    axs[i, 0].imshow(preprocessed_images[i], cmap='gray')
    axs[i, 0].set_title(f'Preprocessed Image {i+1}')
    axs[i, 0].axis('off')
    
    axs[i, 1].imshow(preprocessed_masks[i], cmap='gray')
    axs[i, 1].set_title(f'Preprocessed Mask {i+1}')
    axs[i, 1].axis('off')
    
    axs[i, 2].imshow(segmented_masks_global[i], cmap='gray')
    axs[i, 2].set_title(f'Global Threshold {i+1}')
    axs[i, 2].axis('off')
    
    axs[i, 3].imshow(segmented_masks_adaptive[i], cmap='gray')
    axs[i, 3].set_title(f'Adaptive Threshold {i+1}')
    axs[i, 3].axis('off')
    
    axs[i, 4].imshow(segmented_masks_otsu[i], cmap='gray')
    axs[i, 4].set_title(f'Otsu Threshold {i+1}')
    axs[i, 4].axis('off')
    
    axs[i, 5].imshow(segmented_masks_watershed[i], cmap='gray')
    axs[i, 5].set_title(f'Watershed {i+1}')
    axs[i, 5].axis('off')

plt.tight_layout()
plt.show()


---

<a id='sec_evaluation'></a>

## Evaluation

$\Rightarrow$ Describe the evaluation of your results


### Goals:
* Choose an evaluation method that can compare two binary segmentation masks and computes a numeric score that describes how well these masks match (use for example the Dice score)
* Hint: specify a function `evaluate(mask1, mask2)` that computes the evaluation score(s)
* Compute mean and standard deviation of the scores of the entire dataset

---

<a id='sec_discussion'></a>

## Discussion

Borders
Unfortunately, Otsu and Watershed can't deal well with some images containing black borders around them. I have tried cropping these borders away in the preprocesssing but after investing a lot of time, I didn't manage to create a workable function for this issue.




---

<a id='sec_references'></a>

## References

$\Rightarrow$ Add here references as URLs.

Also declare the usage of **generative AI** here!!



There are many ways how to overlay an image with the mask. Here is one option:

In [ ]:
# Enforce a (3-channel) color image
path_image = "../data/images/neurons-cultured.jpg"
image = cv.imread(path_image, cv.IMREAD_COLOR)
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

# Mask image
path_mask = "../data/images/neurons-cultured-mask.png"
mask = cv.imread(path_mask, cv.IMREAD_GRAYSCALE)

# Create overlay (RGB)
overlay_color = [255, 0, 0]
overlay_alpha = 0.3
overlay = image.copy()
overlay[mask > 0] = overlay_color
overlay = cv.addWeighted(image, 1 - overlay_alpha, overlay, overlay_alpha, 0)

# Display the images next to each other using a convenience function
tools.show_image_chain((image, overlay), titles=("Input", "Overlay"))


In [ ]:
# We could also create contours around the mask and display them
overlay_color = [255, 255, 0]
line_width = 1
contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
image_contours = image.copy()
cv.drawContours(image_contours, contours, -1, overlay_color, line_width)
tools.show_image_chain((image, image_contours), titles=("Input", "Contours"))

An advanced example: We can colorize the different contours with different colors.

Strategy:
- Use connected components to label the different regions using integers 
(every region has a different label)
- Assign a different color to different labels by encoding the label in 
the hue channel (HSV color space!)
- Extract contours from the mask (must be a binary image)
- Merge draw the contours with the colorized labels onto the original image

In [ ]:
# This will contain the result
image_contours = image.copy()

# Compute the "connected components" (= separate objects in the mask)
n_labels, labels = cv.connectedComponents(mask)

# Assign a different color to each label in the hue channel (HSV color space)
hue = np.uint8(150*labels/np.max(labels))
blank = 255*np.ones_like(hue)
labels = cv.merge([hue, blank, blank])

# Convert from HSV color space to RGB
labels = cv.cvtColor(labels, cv.COLOR_HSV2RGB)
# Set the background label (labels==0) to black
labels[labels==0] = 0

# Create a mask of the contours
line_width = 1
contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
mask_contours = cv.drawContours(np.zeros_like(mask), contours, -1, 255, line_width)

# Assign the colored labels only along the contours
image_contours[mask_contours>0] = labels[mask_contours>0]

# Display the result
tools.show_image_chain((image, image_contours), titles=("Input", "Labeled contours"))

### How to convert a Jupyter notebook into a PDF:

- Don't forget to save this notebook before converting!
- Install nbconvert: `pip install nbconvert`
- Convert the notebook into a HTML file: `jupyter nbconvert --to html main.ipynb`  
  The file will be saved in the same folder as this Jupyter notebook
- Open the HTML in a browser and print (or save) it as a PDF
- Recommendation: If you use the Opera browser, you can save the HTML as single-page PDF. This looks the best!

In [ ]:
# Make sure you save this notebook, otherwise the HTML 
# output will not contain the latest version!!

# Make sure you have nbcovnert installed
!pip install nbconvert --quiet
# Save the notebook as HTML
!jupyter nbconvert --to html main.ipynb
